In [9]:
import os
import pickle 
import pandas as pd
import jieba
from zhon.hanzi import punctuation
import re
import googletrans
from pprint import pprint
from googletrans import Translator

## 1. 載入Corpus

In [10]:
path = 'D:\Lala\sa_chinese_project\PTT_Corpus_Data'
os.chdir(path)

data = pd.read_pickle('Finance-1-1332.pkl')
data_extract = data[['content', 'date']]
data_extract['content'] = data_extract['content'].astype(str)
data_extract .head()

C:\Users\lala0\AppData\Local\Temp\ipykernel_14492\2370601725.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_extract['content'] = data_extract['content'].astype(str)


,content,date
0,Finance 板申請開板連署通過，即日起開板試閱 試閱期間三個月，期滿後將再評估是否轉為正...,Mon Aug 11 21:13:51 2008
1,賀finance開板 祝此板以後大有可為 大家都可以分享自己的心得 然後在這邊能得到自己想知...,Mon Aug 11 21:48:50 2008
2,剛剛不知道是不是眼殘...在分類還找不到路進來 = = 最後是按s 打看版名才進來的 希望人...,Mon Aug 11 22:25:32 2008
3,身為金融業的一份子一定要進來恭喜的 好開心有自己的地方可以討論...XD 希望人氣越來越高呀,Mon Aug 11 22:26:32 2008
4,請大家po文自介一下吧 感謝大家,Mon Aug 11 22:50:28 2008


### preprocess

In [11]:
def preprocess(text):
    for i in punctuation:
        text = text.replace(i, '')
    text = re.sub('[\d]','',text) # [0-9]
    
    text = re.sub(r'[.,"\'-?:!;]', '', text)
    
    text = re.sub('[a-zA-Z]','', text)
    
    text = text.replace("\n", "")
    text = text.replace(" ", "")
    return text

def tokenization(text):
    seg_list = jieba.cut(text, cut_all=False)
    return list(seg_list)

In [12]:
data_extract['prepreocessed'] = data_extract['content'].map(preprocess)
data_extract['tokens'] = data_extract['prepreocessed'].map(tokenization)

del data_extract['content']
del data_extract['prepreocessed']

C:\Users\lala0\AppData\Local\Temp\ipykernel_14492\2526939230.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_extract['prepreocessed'] = data_extract['content'].map(preprocess)
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lala0\AppData\Local\Temp\jieba.cache
Loading model cost 0.584 seconds.
Prefix dict has been built successfully.
C:\Users\lala0\AppData\Local\Temp\ipykernel_14492\2526939230.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_extract['t

In [13]:
data_extract

,date,tokens
0,Mon Aug 11 21:13:51 2008,"[板, 申請, 開板, 連署, 通過, 即日起, 開板試, 閱試, 閱期間, 三個, 月期,..."
1,Mon Aug 11 21:48:50 2008,"[賀開板, 祝, 此板, 以後大有, 可為, 大家, 都, 可以, 分享, 自己, 的, 心..."
2,Mon Aug 11 22:25:32 2008,"[剛剛, 不, 知道, 是不是, 眼殘, 在, 分類, 還找, 不到, 路, 進來, 最後是..."
3,Mon Aug 11 22:26:32 2008,"[身為, 金融, 業的, 一份, 子, 一定, 要, 進來, 恭喜, 的, 好, 開心, 有..."
4,Mon Aug 11 22:50:28 2008,"[請, 大家, 文自介, 一下, 吧, 感謝, 大家]"
...,...,...
26509,Sun Aug 7 22:22:07 2022,"[大家, 好, 我, 是, 國立, 臺, 灣大學, 心理, 學, 研究所, 工商, 組碩班,..."
26510,Mon Aug 8 21:45:20 2022,"[大家, 好, 今天, 台銀, 筆試, 放榜, 我, 是, 考中彰, 投區, 的, 催收, ..."
26511,Thu May 13 20:21:32 2010,"[開放此, 篇, 推文, 交流, 推文請, 詳盡, 告知, 條件, 必, 注意, 對方, 是..."
26512,Mon Jul 5 23:43:55 2010,"[格式, 此後買, 賣文, 除非, 連續, 狂者, 我會, 水桶, 文者, 之外, 我會, ..."


## 2. 載入翻譯word 

In [8]:
name = "D:/Lala/sa_chinese_project/NTUSD-Fin/NTUSD_Fin_sentiment.json"
df_score = pd.read_json(name, encoding="utf-8")
# df_score = df[['token', 'market_sentiment']]
df_score

,token,market_sentiment
0,筆記,1.081211
1,機械手,1.221692
2,吸,-1.507409
3,車庫,1.069689
4,艾滋病,0.922132
...,...,...
8441,☕,1.141662
8442,😀,-0.644559
8443,👍,1.100400
8444,👆,1.059200


In [41]:
df_score.describe()

,market_sentiment
count,8446.000000
mean,0.570526
std,0.950257
min,-4.418490
25%,0.325545
50%,1.010188
75%,1.128583
max,1.221692


In [33]:
def Average(lst):
    return sum(lst) / len(lst)

a = list(df_score['token'])
def sa_print(tokens):
    score = 0
    count = 0
    for i in tokens:
        if i in a:
            tmp = list(df_score[df_score['token'] == i]['market_sentiment'])
            print(i, tmp)
            avg = Average(tmp)
            count += 1
            score += avg
            print(count, avg, score)

    if(count == 0): return 0
    else: return score/count

def sa(tokens):
    score = 0
    count = 0
    for i in tokens:
        if i in a:
            tmp = list(df_score[df_score['token'] == i]['market_sentiment'])
            avg = Average(tmp)
            count += 1
            score += avg

    if(count == 0): return 0
    else: return score/count

In [25]:
print(a)

['筆記', '機械手', '吸', '車庫', '艾滋病', '放', '睡覺', '便士', '證明', '三倍', '是的', '設計的', '倡議', '不斷地', '溫暖的', '癮', '開始', '思科', '再融資', '升起', '基準', '三倍', '清單', '多', '轉變', '合法的', '長', '基本上', '避免', '夢想家', '化合物', '雜草', '他媽的', '圖標', '顱骨', '希望', '停滯不前', '口袋妖怪', '印度', '否認', '隱私', '翅膀', '得克薩斯州', '知識分子', '隊友', '完成', '隱', '膠水', '跟踪', '厄運', '衝擊', '分鐘', '高跟鞋', '違反', '考慮', '腎臟', '封頂', '比特幣', '可交易的', '鬼鬼祟祟', '通訊', '主導的', '配料', '編輯', '墨西哥', '灰度', '昂貴的', '已到期', '沃倫', '接收', '殺戮', '聲稱', '討論過', '明天', '有針對性的', '遵守', '反應', '小的', '聯繫', '提供', '保護', '雲', '遊樂設施', '阿西亞', '部', '上升', '發明的', '弱點', '一樣', '除非', '雜貨店', '颶風', '細繩', '托運人', '每月', '絲毫', '摩爾', '無論如何', '上訴', '收穫', '魔法', '青菜', '圖片', 'rcc', '建議', '星期三', '明天', '虐待', '風暴', '谷歌', '鎖起來', '解釋', '指標', 'RSLS', '日間交易', '飆升', '金條', '鐵', '里程碑', '刪除', '拳頭', '殺死', '強調', '金錢', '如果', '基石', '有利', '崩潰', '準備好了', '出現', '喵喵', '希望', '古納', '輪胎', '投降', '查詢', '小說', '地雷', '擠', '繁榮', '在線的', '希望', '獨角獸', '似乎', '害怕', '埋伏者', '慣於', '擠壓的', '意識到', '攀登', '持有者', '觸及', '裂縫', '增加', '頂峰', '失敗', '招聘', '悲慘地', '發

### example

print(count, avg, score)

In [34]:
sa_print(data_extract['tokens'][0])

申請 [-0.2113598529]
1 -0.2113598529 -0.2113598529
通過 [0.326296933, 0.5936319317000001, -0.04555096, 0.20598780700000002]
2 0.270091427925 0.05873157502500001
評估 [1.1285827566, 0.9737646475, 0.9586577551000001, 1.0372675898]
3 1.0245681872499999 1.0832997622749998
正式 [-0.5523967708]
4 -0.5523967708 0.5309029914749998
也 [1.1961570689, 1.0863403081]
5 1.1412486885000002 1.6721516799749998
希望 [1.1658389262, 1.221692161, 0.5248959980000001, -0.31513854900000005, 1.1621644596, 1.1221564874, -0.8211970223, -3.8115636372, 0.9491048192, 1.1195939733]
6 0.23175476162000005 1.9039064415949998
給予 [0.9378991950000001, 0.051993415800000005, -0.3304848474]
7 0.21980258780000006 2.123709029395
支持 [-1.9224468153, -0.7597964776, 1.0213935105, -0.207808914, 1.1147769570000001]
8 -0.15077634787999994 1.9729326815150001
金融 [-0.699349953, 0.3361279424, -0.6945688549]
9 -0.3525969551666666 1.6203357263483336
類別 [1.029047083]
10 1.029047083 2.6493828093483334
請 [1.221692161, 0.7126912938000001, 0.4457524335, 0

希望 [1.1658389262, 1.221692161, 0.5248959980000001, -0.31513854900000005, 1.1621644596, 1.1221564874, -0.8211970223, -3.8115636372, 0.9491048192, 1.1195939733]
231 0.23175476162000005 118.43903178947322
可以 [0.509022857, 1.1651086326]
232 0.8370657448000001 119.27609753427322
金融 [-0.699349953, 0.3361279424, -0.6945688549]
233 -0.3525969551666666 118.92350057910656
同行 [1.0773022516, 1.0892418649]
234 1.08327205825 120.00677263735656
交換 [1.221692161, 0.0564915692, 0.9085342757, 1.1602916162999999]
235 0.83675240555 120.84352504290656
來源 [0.9992997396000001]
236 0.9992997396000001 121.84282478250657
金融 [-0.699349953, 0.3361279424, -0.6945688549]
237 -0.3525969551666666 121.49022782733991
真的 [0.0916100842, 1.1392300008, 1.0411199153, 0.5176569881]
238 0.6974042471 122.1876320744399
需要 [0.8997640661, 0.356427504, 0.6246357383000001, 0.0444285455, 0.7112980585, 1.0152412835, 1.0796731561]
239 0.6759240502857143 122.86355612472562
地方 [0.30868326020000003]
240 0.30868326020000003 123.17223938492

0.515473701886171

### example2
print(count, avg, score)

In [35]:
sa_print(data_extract['tokens'][10])

錯過 [-2.5227572803, 0.12042350260000001, 0.2544776533]
1 -0.7159520414666667 -0.7159520414666667
時間 [1.221692161, 1.221692161, 0.7622605423000001]
2 1.0685482881 0.3525962466333332
可以 [0.509022857, 1.1651086326]
3 0.8370657448000001 1.1896619914333333
有 [1.1392300008, 0.026799884200000002, 1.1147769570000001, 0.9737646475, 1.143689649]
4 0.8796522277 2.069314219133333
金融 [-0.699349953, 0.3361279424, -0.6945688549]
5 -0.3525969551666666 1.7167172639666666


0.34334345279333334

In [37]:
sa_print(data_extract['tokens'][2])

不 [-1.6678739882, -1.5552728147, 1.1513028331, 1.0135839656]
1 -0.2645650010499999 -0.2645650010499999
知道 [-0.1171953511, 0.3170191194, 0.9737646475]
2 0.3911961386 0.1266311375500001
在 [0.43143903, 1.1285827566, -1.2528364888999999, 1.0152412835, -0.1322213293, 1.1429150476]
3 0.38885338325 0.5154845208000001
路 [0.5003263328, 0.1756632702]
4 0.33799480150000005 0.8534793223000001
進來 [-1.0423836809]
5 -1.0423836809 -0.18890435859999988
按 [0.0785070327]
6 0.0785070327 -0.11039732589999987
打 [0.6941600109, 0.2548758343, 1.118598668, 1.1062149435]
7 0.7934623641749999 0.6830650382750001
進來 [-1.0423836809]
8 -1.0423836809 -0.3593186426249999
的 [1.1285827566, 0.0068957254, 1.0841886372, 1.0961612789, -1.5309777870999999]
9 0.3569701222 -0.0023485204249998892
希望 [1.1658389262, 1.221692161, 0.5248959980000001, -0.31513854900000005, 1.1621644596, 1.1221564874, -0.8211970223, -3.8115636372, 0.9491048192, 1.1195939733]
10 0.23175476162000005 0.22940624119500017
高 [0.8591220816]
11 0.8591220816 1

0.2118656755449123

## 3. 使用function

In [26]:
data_extract['score'] = data_extract['tokens'].map(sa)

C:\Users\lala0\AppData\Local\Temp\ipykernel_14492\3775716914.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_extract['score'] = data_extract['tokens'].map(sa)


In [40]:
fliter = (data_extract["score"] < 0)
data_extract[fliter]

,date,tokens,score
274,Thu Oct 2 18:48:06 2008,"[想, 請問, 這樣, 穿會, 不會, 不, 正式, 只有, 一件, 黑色, 西裝, 外套,...",-0.256081
344,Thu Oct 16 11:06:59 2008,"[想, 請, 問是, 幾號, 因為, 想, 知道, 大概, 幾月, 幾號, 以後受, 訓完,...",-0.064010
557,Sat Nov 29 13:09:16 2008,"[下列, 何者, 不, 需提, 存款, 準備, 金公庫, 存款, 支票, 存款, 可轉, 定...",-1.478759
1065,Wed Apr 15 00:30:42 2009,"[台銀證, 徵才, 職等, ~, 職等, 詳金, 研院, 網站, _, ~, 報名]",-0.134367
1934,Fri Oct 2 21:52:47 2009,"[以存, 貨動機, 解釋, 交易, 性實質, 貨幣, 需求, 可知, 實質, 所得, 上生,...",-0.247839
...,...,...,...
24796,Tue Oct 20 14:04:59 2020,"[~, _, [, ], [, ], [, ], [, ], [, ]]",-0.723138
25149,Tue Mar 9 22:21:45 2021,"[第二, 題, 某, 公司, 資產, 總額, 負債, 平均, 利率, %, 若, 總資產, ...",-0.189577
25296,Fri Apr 30 14:05:28 2021,"[今年, 的, 企金人員, 怎麼, 都, 沒人, 討論, 上星期, 好像, 才, 剛考, 完...",-0.056677
25334,Thu May 13 21:38:29 2021,[_],-0.723138
